In [5]:
import pandas as pd
import sys
sys.path.append('../pipeline_v2/')
import main

/opt/homebrew/Caskroom/miniconda/base/envs/factchecker/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [22]:
model = 'gemini'
texts = []
sample_df = pd.read_pickle(f'results_v2_{model}.pkl')
for i in range(len(sample_df)):
    row = sample_df.iloc[i]
    results = row[f'{model}_pipeline_results']
    result = results[0]
    claims = result['claims']
    statement_text = f"On {row['statement_date']}, {row['statement_originator']} claimed: {row['statement']}"
    reasoning = result['reasoning'].replace('\n', ' ')
    
    text_printed = f"Statement Evaluation {i + 1}\n"
    text_printed += f"Statement: {statement_text}\n"
    text_printed += f"Overall Verdict: {row['pipeline_pass3_verdict']}\n"
    text_printed += f"Overall Confidence: {result['confidence']}\n"
    text_printed += f"Overall Reasoning: {reasoning}\n\n"

    text_printed += f"Claims Extracted & Independently Verified:\n"

    for i, claim in enumerate(claims):
        text_printed += f"Claim {i + 1}/{len(claims)}: {claim.text}\n"
        text_printed += f"Verdict: {claim.verdict}\n"
        text_printed += f"Confidence: {claim.confidence}\n"
        text_printed += f"Reasoning: {claim.reasoning}\n"

        for component in claim.components:
            answer_text = component.answer.text
            answer_text = answer_text.replace('\n', ' ')
            text_printed += f"Question: {component.question}\n"
            text_printed += f"Answer: {component.answer.text}\n"

            if component.answer.citations:
                text_printed += f"Explicit citations by the model:\n"
                for j, citation in enumerate(component.answer.citations, 1):
                    if citation: 
                        text_printed += f"[{j}] {citation.snippet}\n"
                        text_printed += f"{citation.source_title}  - {citation.source_url}\n"
            
            text_printed += '\n'
            if component.answer.retrieved_docs:
                text_printed += f"Documents used to synthesize answer (implicit citations):\n"
                for k, doc in enumerate(component.answer.retrieved_docs, 1):
                    text_printed += f"[{k}] {doc.content if doc.content else ''}\n"
                    text_printed += f"{doc.metadata.get('title', '') if doc.metadata else ''} - {doc.metadata.get('url', '') if doc.metadata else ''}\n"
            text_printed += '\n'

    texts.append(text_printed)

In [23]:
print(texts[4])

Statement Evaluation 5
Statement: On December 01, 2024, Jay Rothman claimed: “The Universities of Wisconsin are 43rd out of 50 states in the nation, in terms of public support for our universities. The $855 million (budget request) gets us up to average.”
Overall Verdict: MOSTLY TRUE
Overall Confidence: 0.7
Overall Reasoning: The first claim, that Jay Rothman stated Wisconsin universities rank 43rd out of 50 in public support, is mostly true.  Sources confirm Wisconsin's low ranking in higher education funding, although the exact ranking and the precise date of Rothman's statement require further verification. The second claim, that an $855 million budget increase would bring Wisconsin to the national average, is unverifiable. While the budget request is documented, whether it would actually achieve the stated goal is not supported by the evidence.  Since one claim is mostly true and the other unverifiable, the overall statement leans towards being mostly true, but the lack of definiti